In [15]:
from transformers import pipeline

In [16]:
# More models in the model hub.
model_name = "openai/clip-vit-large-patch14-336"
classifier = pipeline("zero-shot-image-classification", model = model_name)

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


#### Label wrt Protocol Procedure

In [79]:
# cardiac arrest, chest pain?
cv_ecg_12_lead = ["ECG Monitor","ECG 12 lead placement"]
cv_defibrillation_manual = ["Defibrillator","Defib pads attachment"]
cpr_manual = []

# respiratory distress
resp_assist_ventilation_bvm_via_mask = ["BVM Mask", "Attaching BVM Mask"]
resp_nebulizer_therapy = ["Attaching Nebulizer", "Nebulizer Mask"]

# general
iv_start = ["Inserting IV to arm","Inserting IV to leg"]

In [86]:
image_to_classify = "./images/iv start - extremity vein/iv_arm_1.jpeg"

labels_for_classification = iv_start+  resp_assist_ventilation_bvm_via_mask + cv_ecg_12_lead + resp_nebulizer_therapy
scores = classifier(image_to_classify, 
                    candidate_labels = labels_for_classification)

In [87]:
scores

[{'score': 0.7846152186393738, 'label': 'Inserting IV to arm'},
 {'score': 0.2148704081773758, 'label': 'Inserting IV to leg'},
 {'score': 0.00043944327626377344, 'label': 'Attaching BVM Mask'},
 {'score': 5.8845485909841955e-05, 'label': 'ECG 12 lead placement'},
 {'score': 1.0952401680697221e-05, 'label': 'ECG Monitor'},
 {'score': 4.087542492925422e-06, 'label': 'Attaching Nebulizer'},
 {'score': 7.775275321364461e-07, 'label': 'BVM Mask'},
 {'score': 2.0235295039583434e-07, 'label': 'Nebulizer Mask'}]